# Note
The Sagemaker `local` mode works slightly differently with the one which live on cloud. Especially the `load_model` function may have to be implemented differently.

## 0. Prepare

In [ ]:
from sagemaker.local import LocalSession

sagemaker_session = LocalSession()
sagemaker_session.config = {'local': {'local_code': True}}

In [2]:
# Make a dummy file for the model
import os
import tarfile
with open("model.txt", "w") as f:
    f.write("I'm model 1.")

with tarfile.open("model1.tar.gz","w:gz") as tar:
    for file in ["model.txt"]:
        tar.add(os.path.basename(file))

os.remove("model.txt")

In [3]:
import sagemaker
model1 = sagemaker.model.Model(
    image_uri="demo-sagemaker-template-bo-test:latest", # The built image on your local machine
    model_data="file://model1.tar.gz",
    sagemaker_session=sagemaker_session,
    role='arn:aws:iam::318775028588:role/sagemaker_Prototype'
)

## 1. Deploy 

In [4]:
from sagemaker.predictor import Predictor
model1.deploy(initial_instance_count=1, instance_type="local", predictor_cls=Predictor)

Attaching to 9okx4z1i9f-algo-1-tr7h2
9okx4z1i9f-algo-1-tr7h2 | Warning: MMS is using non-default JVM parameters: -XX:-UseContainerSupport
9okx4z1i9f-algo-1-tr7h2 | 2021-08-02 18:35:37,748 [INFO ] main com.amazonaws.ml.mms.ModelServer - 
9okx4z1i9f-algo-1-tr7h2 | MMS Home: /usr/local/lib/python3.6/dist-packages
9okx4z1i9f-algo-1-tr7h2 | Current directory: /
9okx4z1i9f-algo-1-tr7h2 | Temp directory: /tmp
9okx4z1i9f-algo-1-tr7h2 | Number of GPUs: 0
9okx4z1i9f-algo-1-tr7h2 | Number of CPUs: 12
9okx4z1i9f-algo-1-tr7h2 | Max heap size: 5346 M
9okx4z1i9f-algo-1-tr7h2 | Python executable: /usr/bin/python3
9okx4z1i9f-algo-1-tr7h2 | Config file: /etc/sagemaker-mms.properties
9okx4z1i9f-algo-1-tr7h2 | Inference address: http://0.0.0.0:8080
9okx4z1i9f-algo-1-tr7h2 | Management address: http://0.0.0.0:8080
9okx4z1i9f-algo-1-tr7h2 | Model Store: /.sagemaker/mms/models
9okx4z1i9f-algo-1-tr7h2 | Initial Models: ALL
9okx4z1i9f-algo-1-tr7h2 | Log dir: /logs
9okx4z1i9f-algo-1-tr7h2 | Metrics dir: /logs
9

## 2. Inference

In [5]:
from sagemaker.serializers import NumpySerializer, JSONSerializer, CSVSerializer
from sagemaker.deserializers import NumpyDeserializer, JSONDeserializer

In [6]:
predictor = Predictor(
    endpoint_name=model1.endpoint_name,
    sagemaker_session=model1.sagemaker_session,
    serializer=JSONSerializer(),
    # serializer=JSONSerializer(),
)

In [7]:
data = {
    "text":"Hello world! I like pizza! How about you?",
    "n_sent":1
}

str(data)

import json
# json.dumps(data)

response = predictor.predict(json.dumps(data), target_model="model1.tar.gz")

9okx4z1i9f-algo-1-tr7h2 | 2021-08-02 18:35:41,884 [INFO ] W-model-12-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Hi I'm called once
9okx4z1i9f-algo-1-tr7h2 | 2021-08-02 18:35:41,884 [INFO ] W-9000-model com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 3
9okx4z1i9f-algo-1-tr7h2 | 2021-08-02 18:35:41,884 [INFO ] W-model-12-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - reruned
9okx4z1i9f-algo-1-tr7h2 | 2021-08-02 18:35:41,884 [INFO ] W-model-12-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - reruned here 
9okx4z1i9f-algo-1-tr7h2 | 2021-08-02 18:35:41,884 [INFO ] W-9000-model ACCESS_LOG - /172.18.0.1:49650 "POST /invocations HTTP/1.1" 200 5
